### Import the necessary libraries

In [46]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

### 1. Cleaning 

#### Read the contents on the url page into a pandas dataframe.

In [39]:
df=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

#### We need only the first table's contents into our dataframe.

In [40]:
df=df[0]

#### Check the contents of the table.

In [41]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Eliminate the rows where the borough column has value equals to not assigned.

In [42]:
df=df[df['Borough']!='Not assigned']

In [47]:
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


#### Check if there are any rows for which the Neighborhood column values were not assigned.

In [48]:
df[df['Neighborhood']=='Not assigned']

,Postal Code,Borough,Neighborhood


Since there are no rows, we don't have to assign to any rows the same value as Borough.

#### If there are any rows with the same postal code, then we are combining them and joining with ',' to form a single row.

In [50]:
df.groupby('Postal Code')['Neighborhood'].apply(','.join).reset_index()

,Postal Code,Neighborhood
0,M1B,"Malvern, Rouge"
1,M1C,"Rouge Hill, Port Union, Highland Creek"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae
...,...,...
98,M9N,Weston
99,M9P,Westmount
100,M9R,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,"South Steeles, Silverstone, Humbergate, Jamest..."


#### Shape of the dataframe

In [49]:
df.shape

(103, 3)